In [1]:
import fasttext

In [3]:
import torch
from torchsummary import summary
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.functional as F
from sentence_transformers import SentenceTransformer
from transformers import BertForSequenceClassification,BertTokenizer

from tqdm import tqdm



import pandas as pd
import numpy as np
import os

# StarWars



In [2]:
base_dir = "../data/raw/StarWarsEpisodes"

In [3]:
folder_ep4 = os.path.join(base_dir,"SW_EpisodeIV.txt")
folder_ep5 = os.path.join(base_dir,"SW_EpisodeV.txt")
folder_ep6 = os.path.join(base_dir,"SW_EpisodeVI.txt")

In [4]:
df_ep4 = pd.read_csv(folder_ep4, sep =' ', header=0, escapechar='\\')
df_ep5 = pd.read_csv(folder_ep5, sep =' ', header=0, escapechar='\\')
df_ep6 = pd.read_csv(folder_ep6, sep =' ', header=0, escapechar='\\')


In [5]:
df_ep4


,character,dialogue
1,THREEPIO,Did you hear that? They've shut down the main...
2,THREEPIO,We're doomed!
3,THREEPIO,There'll be no escape for the Princess this time.
4,THREEPIO,What's that?
5,THREEPIO,I should have known better than to trust the l...
...,...,...
1006,LUKE,"Oh, no!"
1007,THREEPIO,"Oh, my! Artoo! Can you hear me? Say somethi..."
1008,TECHNICIAN,We'll get to work on him right away.
1009,THREEPIO,"You must repair him! Sir, if any of my circui..."


In [6]:
Y = pd.concat([df_ep4['character'],df_ep5['character'],df_ep6['character']]).tolist()
X = pd.concat([df_ep4['dialogue'],df_ep5['dialogue'],df_ep6['dialogue']]).tolist()

In [7]:
labels = np.unique(Y)
label_count = [sum(i == np.array(Y)) for i in labels]
for i,(a,b) in enumerate(zip(labels,label_count)):
    if b < 10:
        labels[i] = "Other"
labels = np.unique(labels)


In [8]:
labels

array(['ACKBAR', 'BEN', 'BIGGS', 'COMMANDER', 'CREATURE', 'EMPEROR',
       'GOLD LEADER', 'HAN', 'JABBA', 'LANDO', 'LEIA', 'LUKE', 'OFFICER',
       'OWEN', 'Other', 'PIETT', 'RED LEADER', 'RIEEKAN', 'TARKIN',
       'THREEPIO', 'TROOPER', 'VADER', 'WEDGE', 'YODA'], dtype='<U30')

In [9]:
char2ind = {i:j for i,j in zip(labels,range(len(labels)))}
ind2char = {j:i for i,j in zip(labels,range(len(labels)))}

In [10]:
new_x = X.copy()
new_y = []

In [11]:
for idx in range(len(new_x)):
    
    if Y[idx] in labels:
        label_point = char2ind[Y[idx]]
    else:
        label_point = char2ind["Other"]
    new_y.append(label_point)

In [17]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", problem_type="multi_label_classification")
model.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [18]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

In [19]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

In [1]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load the tokenizer and the model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Define your inputs and labels
text_batch = ["I love Pixar.", "I don't care for Pixar."]
encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']
labels = torch.tensor([1, 0])  # The ground truth labels for your examples

# Forward pass through the model
outputs = model(input_ids, attention_mask=attention_mask)
logits = outputs.logits

# Compute the loss
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(logits, labels)

# Backward pass and optimization step
loss.backward()
optimizer.step()


: 

: 

In [14]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
sentence_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", problem_type="multi_label_classification")

with torch.no_grad():
    pre_models = ['bert-base-nli-mean-tokens',
                  'all-mpnet-base-v2',
                  "multi-qa-mpnet-base-dot-v1"]
   # sentence_model = SentenceTransformer(pre_models[2])#SentenceTransformer('bert-base-nli-mean-tokens')
    encoded_inputs = tokenizer.batch_encode_plus(X, padding=True, truncation=True, return_tensors='pt')
    logits = sentence_model(**encoded_inputs)[0]

    new_x = sentence_model.encode(X)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

: 

: 

In [2]:
encoded_inputs = tokenizer.batch_encode_plus(X, padding=True, truncation=True, return_tensors='pt')


NameError: name 'tokenizer' is not defined

In [ ]:
# char_names = movie_lines.iloc[:,0]
# movie_names = movie_lines.iloc[:,1]
# char_names = np.unique(list(set(char_names.values)))
# movie_names = np.unique(list(set(movie_names.values)))

In [1]:
class CustomStarWarsDataset(Dataset):
    def __init__(self, X, Y,transform=None, target_transform=None):
        self.X = X
        self.Y = Y

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        data_point = self.X[idx]

        label_point = self.Y[idx]

        # print("data_point is:",data_point)
        # print("label_point is:",label_point)
        # 768 
        # print(sentence_encoded,label_point)
        return data_point, label_point

NameError: name 'Dataset' is not defined

In [ ]:
len(Y)

In [ ]:
class BertSentenceClassifier(nn.Module):
    def __init__(self):
        super(BertSentenceClassifier, self).__init__()
        self.dropout_rate = .1
        self.lin1 = nn.Linear(768,256)
        self.lin_layers = nn.ModuleList([nn.Linear(256,256) for i in range(4)])


        self.lin2 = nn.Linear(256, len(labels))



    def forward(self, data):

        x = nn.functional.relu(self.lin1(data))
        x = nn.functional.dropout(x,self.dropout_rate)
        for i in self.lin_layers:
            x = nn.functional.relu(i(x))
            x = nn.functional.dropout(x,self.dropout_rate)

        x = self.lin2(x)

        x = nn.functional.softmax(x,dim = 1)

        
        return x

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split

In [ ]:
# x_train,x_test,y_train,y_test = train_test_split(new_x,new_y)

In [ ]:
# model = RandomForestClassifier(max_leaf_nodes=25)
# model.fit(x_train,y_train)

In [ ]:
# model.score(x_train,y_train),model.score(x_test,y_test)

In [ ]:
train_dataset = CustomStarWarsDataset(new_x,new_y)
train_set, val_set = torch.utils.data.random_split(train_dataset, [2000, 523])

train_loader = DataLoader(train_set,batch_size=32,shuffle=True,drop_last=True)
val_loader = DataLoader(val_set,batch_size=32,shuffle=True,drop_last=True)

In [ ]:
model = BertSentenceClassifier()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
def accuracy(preds, labels):
    _, predictions = torch.max(preds, dim=1)
    correct = (predictions == labels).sum().item()
    return correct / len(labels)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

In [ ]:
def train(num_epochs):
    global model
    model = model.to(device)

    for epoch in range(num_epochs):
    
        #loop = tqdm(train_loader, total=len(train_loader))
        model.train()
        running_loss = 0.0
        running_accuracy = 0.0
        num_batches = 0
        for text, author_labels in train_loader:  # Assuming data_loader is set up to provide batches of data
            data = text.to(device)
            author_labels = author_labels.to(device)
            optimizer.zero_grad()

            data = model(data)
            # print(data.shape)
            # print("data and author labels ::::: ",data,author_labels)
            loss = nn.CrossEntropyLoss()(data,author_labels)
            acc = accuracy(data, author_labels)

            loss.backward()
            running_loss += loss.item()
            running_accuracy += acc
            num_batches += 1

            optimizer.step()
            # loop.set_description(f"Epoch [{epoch + 1}/{num_epochs}]")
            # loop.set_postfix(loss=loss.item())

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        val_running_accuracy = 0.0
        val_num_batches = 0
        with torch.no_grad():  # Disable gradient calculation during validation
            for val_text, val_labels in val_loader:
                val_data = val_text.to(device)
                val_labels = val_labels.to(device)

                val_output = model(val_data)
                val_acc = accuracy(val_output, val_labels)

                val_running_accuracy += val_acc
                val_num_batches += 1
        avg_loss = running_loss / num_batches
        avg_accuracy = running_accuracy / num_batches
        val_avg_accuracy = val_running_accuracy / val_num_batches
        if epoch%100 ==1: 
            print(f"Validation Accuracy: {val_avg_accuracy:.4f} Accuracy: {avg_accuracy:.4f}")
            
            print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")


In [ ]:
train(1000)

In [ ]:
train(1000)

In [ ]:
train(1000)